In [1]:
using BenchmarkTools
using Distributed
using TimerOutputs

In [16]:
X = rand(54, 581012)*100;

In [6]:
function get_gains!(X, current_values, idxs, gains)
    Threads.@threads for i in eachindex(idxs)
        s = 0.0
        for j in eachindex(current_values)
            @inbounds s += @fastmath sqrt(current_values[j] + X[j, idxs[i]])
        end
        gains[i] = s
    end
end;

In [7]:
function calculate_gains!(X, gains, current_values, idxs, current_concave_value_sum)
    get_gains!(X, current_values, idxs, gains)
    
    gains .-= current_concave_value_sum
    return gains
end;

In [17]:
@btime begin
    d, n = size(X);
    idxs = 1:n |> collect

    current_values = zeros(Float64, d)
    current_concave_values = sqrt.(current_values)
    current_concave_values_sum = sum(current_concave_values)

    gains = zeros(Float64, n);
    calculate_gains!(X, gains, current_values, idxs, current_concave_values_sum);
end

  9.130 ms (96 allocations: 8.87 MiB)


581012-element Vector{Float64}:
 322.32773270652376
 382.9562070864021
 365.5485152912516
 365.75770572584594
 383.18833649929616
 362.62603349700396
 355.5093914170371
 363.158694415829
 325.3034941715409
 360.8704474503573
 378.9675290543217
 364.85290335519625
 330.77501785921936
   ⋮
 369.40183610573973
 358.3192580535032
 354.7823600160897
 351.995954338707
 369.2663740409964
 346.44615323506355
 362.4266694368385
 349.96662796009343
 349.1472282838731
 365.2317912471448
 371.140435896119
 349.28805359958784